_한국어로 기계번역됨_


# 과거 그래프 상태 보기 및 업데이트 방법

!!! 팁 "필수 조건"

    이 가이드는 다음 개념에 대한 이해가 있다고 가정합니다:

    * [타임 트래블](../../../concepts/time-travel)
    * [브레이크포인트](../../../concepts/breakpoints)
    * [랭그래프 용어사전](../../../concepts/low_level)

그래프를 [체크포인트](../../persistence)하기 시작하면, 언제든지 에이전트의 상태를 쉽게 **가져오거나** **업데이트**할 수 있습니다. 이는 몇 가지 가능성을 허용합니다:

1. 중단 시 사용자에게 상태를 보여주어 행동을 수락하게 할 수 있습니다.
2. 문제를 재현하거나 회피하기 위해 그래프를 **되감기** 할 수 있습니다.
3. 상태를 **변경**하여 에이전트를 더 큰 시스템에 통합하거나 사용자가 에이전트의 행동을 더 잘 제어할 수 있도록 할 수 있습니다.

이 기능을 위해 사용되는 주요 메서드는 다음과 같습니다:

- [get_state](https://langchain-ai.github.io/langgraph/reference/graphs/#langgraph.graph.graph.CompiledGraph.get_state): 대상 구성에서 값을 가져옵니다.
- [update_state](https://langchain-ai.github.io/langgraph/reference/graphs/#langgraph.graph.graph.CompiledGraph.update_state): 주어진 값을 대상 상태에 적용합니다.

**참고:** 이 기능을 사용하려면 체크포인터를 전달해야 합니다.

아래는 간단한 예제입니다.


## 설정

먼저 필요한 패키지를 설치해야 합니다.


In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_openai


다음으로, 우리가 사용할 LLM인 OpenAI의 API 키를 설정해야 합니다.


In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")


ANTHROPIC_API_KEY:  ········


<div class="admonition tip">
    <p class="admonition-title">LangGraph 개발을 위한 <a href="https://smith.langchain.com">LangSmith</a> 설정</p>
    <p style="padding-top: 5px;">
        LangSmith에 가입하면 LangGraph 프로젝트의 문제를 신속하게 발견하고 성능을 개선할 수 있습니다. LangSmith를 사용하면 LangGraph로 구축한 LLM 앱을 디버깅, 테스트 및 모니터링하기 위해 추적 데이터를 활용할 수 있습니다. 시작하는 방법에 대한 자세한 내용은 <a href="https://docs.smith.langchain.com">여기</a>에서 읽어보세요.
    </p>
</div>


## 에이전트 구축하기

이제 에이전트를 구축할 수 있습니다. 도구 호출을 수행하는 비교적 단순한 ReAct 스타일의 에이전트를 구축할 것입니다. 우리는 Anthropic의 모델과 가짜 도구(데모 목적으로만 사용)를 사용할 것입니다.


In [42]:
# Set up the tool
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.graph import MessagesState, START
from langgraph.prebuilt import ToolNode
from langgraph.graph import END, StateGraph
from langgraph.checkpoint.memory import MemorySaver


@tool
def play_song_on_spotify(song: str):
    """Play a song on Spotify"""
    # Call the spotify API ...
    return f"Successfully played {song} on Spotify!"


@tool
def play_song_on_apple(song: str):
    """Play a song on Apple Music"""
    # Call the apple music API ...
    return f"Successfully played {song} on Apple Music!"


tools = [play_song_on_apple, play_song_on_spotify]
tool_node = ToolNode(tools)

# Set up the model

model = ChatOpenAI(model="gpt-4o-mini")
model = model.bind_tools(tools, parallel_tool_calls=False)


# Define nodes and conditional edges


# Define the function that determines whether to continue or not
def should_continue(state):
    messages = state["messages"]
    last_message = messages[-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"


# Define the function that calls the model
def call_model(state):
    messages = state["messages"]
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


# Define a new graph
workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.add_edge(START, "agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END,
    },
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("action", "agent")

# Set up memory
memory = MemorySaver()

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable

# We add in `interrupt_before=["action"]`
# This will add a breakpoint before the `action` node is called
app = workflow.compile(checkpointer=memory)


## 에이전트와 상호작용하기

이제 에이전트와 상호작용할 수 있습니다. 테일러 스위프트의 가장 인기 있는 노래를 틀어 달라고 요청해 봅시다:


In [43]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}
input_message = HumanMessage(content="Can you play Taylor Swift's most popular song?")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()


================================ Human Message =================================

Can you play Taylor Swift's most popular song?
================================== Ai Message ==================================
Tool Calls:
  play_song_on_apple (call_uhGY6Fv6Mr4ZOhSokintuoD7)
 Call ID: call_uhGY6Fv6Mr4ZOhSokintuoD7
  Args:
    song: Anti-Hero by Taylor Swift
================================= Tool Message =================================
Name: play_song_on_apple

Succesfully played Anti-Hero by Taylor Swift on Apple Music!
================================== Ai Message ==================================

I've successfully played "Anti-Hero" by Taylor Swift on Apple Music! Enjoy the music!


## 역사 확인

이 스레드의 시작부터 끝까지의 역사를 살펴보겠습니다.


In [44]:
app.get_state(config).values["messages"]


[HumanMessage(content="Can you play Taylor Swift's most popular song?", id='7e32f0f3-75f5-48e1-a4ae-d38ccc15973b'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_uhGY6Fv6Mr4ZOhSokintuoD7', 'function': {'arguments': '{"song":"Anti-Hero by Taylor Swift"}', 'name': 'play_song_on_apple'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 80, 'total_tokens': 102}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_483d39d857', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-af077bc4-f03c-4afe-8d92-78bdae394412-0', tool_calls=[{'name': 'play_song_on_apple', 'args': {'song': 'Anti-Hero by Taylor Swift'}, 'id': 'call_uhGY6Fv6Mr4ZOhSokintuoD7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 80, 'output_tokens': 22, 'total_tokens': 102}),
 ToolMessage(content='Succesfully played Anti-Hero by Taylor Swift on Apple Music!', name='play_song_on_apple', id='43a39ca7-326a-4033-86

In [45]:
all_states = []
for state in app.get_state_history(config):
    print(state)
    all_states.append(state)
    print("--")


StateSnapshot(values={'messages': [HumanMessage(content="Can you play Taylor Swift's most popular song?", id='7e32f0f3-75f5-48e1-a4ae-d38ccc15973b'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_uhGY6Fv6Mr4ZOhSokintuoD7', 'function': {'arguments': '{"song":"Anti-Hero by Taylor Swift"}', 'name': 'play_song_on_apple'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 80, 'total_tokens': 102}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_483d39d857', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-af077bc4-f03c-4afe-8d92-78bdae394412-0', tool_calls=[{'name': 'play_song_on_apple', 'args': {'song': 'Anti-Hero by Taylor Swift'}, 'id': 'call_uhGY6Fv6Mr4ZOhSokintuoD7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 80, 'output_tokens': 22, 'total_tokens': 102}), ToolMessage(content='Succesfully played Anti-Hero by Taylor Swift on Apple Music!', name='play_song_on_a

## 상태 재생

우리는 이러한 상태 중 어떤 상태로도 돌아가서 에이전트를 거기서 다시 시작할 수 있습니다! 도구 호출이 실행되기 직전으로 돌아가 봅시다.


In [46]:
to_replay = all_states[2]


In [47]:
to_replay.values


{'messages': [HumanMessage(content="Can you play Taylor Swift's most popular song?", id='7e32f0f3-75f5-48e1-a4ae-d38ccc15973b'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_uhGY6Fv6Mr4ZOhSokintuoD7', 'function': {'arguments': '{"song":"Anti-Hero by Taylor Swift"}', 'name': 'play_song_on_apple'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 80, 'total_tokens': 102}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_483d39d857', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-af077bc4-f03c-4afe-8d92-78bdae394412-0', tool_calls=[{'name': 'play_song_on_apple', 'args': {'song': 'Anti-Hero by Taylor Swift'}, 'id': 'call_uhGY6Fv6Mr4ZOhSokintuoD7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 80, 'output_tokens': 22, 'total_tokens': 102})]}

In [48]:
to_replay.next


('action',)

이 위치에서 다시 재생하려면 그 구성 정보를 에이전트에 전달하면 됩니다. 단지 도구 호출을 하던 곳에서 바로 재개하는 것임을 주목하세요.


In [49]:
for event in app.stream(None, to_replay.config):
    for v in event.values():
        print(v)


{'messages': [ToolMessage(content='Succesfully played Anti-Hero by Taylor Swift on Apple Music!', name='play_song_on_apple', tool_call_id='call_uhGY6Fv6Mr4ZOhSokintuoD7')]}
{'messages': [AIMessage(content='I\'ve started playing "Anti-Hero" by Taylor Swift on Apple Music! Enjoy the music!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 126, 'total_tokens': 146}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_483d39d857', 'finish_reason': 'stop', 'logprobs': None}, id='run-dc338bbd-d623-40bb-b824-5d2307954b57-0', usage_metadata={'input_tokens': 126, 'output_tokens': 20, 'total_tokens': 146})]}


## 과거 상태에서 분기하기

LangGraph의 체크포인팅을 사용하면 단순히 과거 상태를 재생하는 것 이상의 작업을 할 수 있습니다. 이전 위치에서 분기하여 에이전트가 대체 경로를 탐색하도록 하거나 사용자가 워크플로의 변경 사항을 "버전 관리"할 수 있도록 할 수 있습니다.

특정 시점에서 상태를 수정하는 방법을 보여드리겠습니다. 애플에서 노래를 재생하는 대신 스포티파이에서 재생하도록 상태를 업데이트해 보겠습니다.


In [52]:
# Let's now get the last message in the state
# This is the one with the tool calls that we want to update
last_message = to_replay.values["messages"][-1]


# Let's now update the tool we are calling
last_message.tool_calls[0]["name"] = "play_song_on_spotify"

branch_config = app.update_state(
    to_replay.config,
    {"messages": [last_message]},
)


그런 다음 이 새로운 `branch_config`를 사용하여 변경된 상태에서 여기서 실행을 재개할 수 있습니다. 로그에서 도구가 다른 입력으로 호출된 것을 확인할 수 있습니다.


In [53]:
for event in app.stream(None, branch_config):
    for v in event.values():
        print(v)


{'messages': [ToolMessage(content='Succesfully played Anti-Hero by Taylor Swift on Spotify!', name='play_song_on_spotify', tool_call_id='call_uhGY6Fv6Mr4ZOhSokintuoD7')]}
{'messages': [AIMessage(content='I\'ve started playing "Anti-Hero" by Taylor Swift on Spotify. Enjoy the music!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 125, 'total_tokens': 144}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_483d39d857', 'finish_reason': 'stop', 'logprobs': None}, id='run-7d8d5094-7029-4da3-9e0e-ef9d18b63615-0', usage_metadata={'input_tokens': 125, 'output_tokens': 19, 'total_tokens': 144})]}


대신, 도구를 호출조차 하지 않도록 상태를 업데이트할 수 있습니다!


In [54]:
from langchain_core.messages import AIMessage

# Let's now get the last message in the state
# This is the one with the tool calls that we want to update
last_message = to_replay.values["messages"][-1]

# Let's now get the ID for the last message, and create a new message with that ID.
new_message = AIMessage(
    content="It's quiet hours so I can't play any music right now!", id=last_message.id
)

branch_config = app.update_state(
    to_replay.config,
    {"messages": [new_message]},
)


In [55]:
branch_state = app.get_state(branch_config)


In [56]:
branch_state.values


{'messages': [HumanMessage(content="Can you play Taylor Swift's most popular song?", id='7e32f0f3-75f5-48e1-a4ae-d38ccc15973b'),
  AIMessage(content="It's quiet hours so I can't play any music right now!", id='run-af077bc4-f03c-4afe-8d92-78bdae394412-0')]}

In [57]:
branch_state.next


()

스냅샷이 업데이트되어 이제 다음 단계가 없다는 것을 올바르게 반영하고 있음을 알 수 있습니다.
